In [ ]:
import pandas as pd
from kaggle_secrets import UserSecretsClient
from huggingface_hub import HfApi, HfFolder
import os

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")

# Set the token for use in the huggingface_hub library
HfFolder.save_token(hf_token)

import wandb

# Replace 'your-api-token' with your actual API token from wandb.ai
wandb.login(key='88bc0f92c0138587605140040c6d0ef652bde63d')

In [ ]:
df = pd.read_csv("/kaggle/input/mental-health-corpus/mental_health.csv")

In [ ]:
df

In [ ]:
df = pd.get_dummies(df, columns=["label"])
df["label_0"] = df["label_0"].astype(int)
df["label_1"] = df["label_1"].astype(int)

In [ ]:
X_data = df["text"]
y_data = df[["label_0", "label_1"]]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_data, y_data, test_size=0.3, stratify=y_data, random_state=42
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    #print(examples)
    return tokenizer(str(examples), truncation=True, padding="max_length", max_length=512)

In [ ]:
X_train = X_train.map(preprocess_function)
X_test = X_test.map(preprocess_function)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
%pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "POSITIVE", 1: "NEGATIVE"}
label2id = {"POSITIVE": 0, "NEGATIVE": 1}
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
from datasets import Dataset
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)
X_train = pd.DataFrame(X_train.tolist(), columns=['input_ids', 'attention_mask'], index=X_train.index)
X_test = pd.DataFrame(X_test.tolist(), columns=['input_ids', 'attention_mask'], index=X_test.index)


In [ ]:
X_train

In [ ]:
train_dataset = Dataset.from_dict({
    "input_ids": X_train['input_ids'],
    "attention_mask": X_train['attention_mask'],
    "labels": y_train
})

test_dataset = Dataset.from_dict({
    "input_ids": X_test['input_ids'],
    "attention_mask": X_test['attention_mask'],
    "labels": y_test
})

In [ ]:
#from transformers import DataCollatorWithPadding
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="Job_compatibility_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()